In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
import os

In [3]:
os.listdir('./')

['.config', 'drive', 'sample_data']

In [0]:
os.chdir('./drive')

In [5]:
os.listdir('./')

['.shortcut-targets-by-id', 'My Drive', '.Trash']

In [0]:
os.chdir('./My Drive')

In [7]:
os.listdir('./')

['app', 'drive', 'yuchuli.ipynb']

In [0]:
os.chdir('./drive')

In [9]:
os.listdir('./')

['rnn_7', '图像分类', '基于注意力机制的翻译.ipynb', 'yuyanmoxing']

In [0]:
os.chdir('./yuyanmoxing')

In [11]:
os.listdir('./')

['text8', '__MACOSX', '语言模型']

In [0]:
os.chdir('./text8')

In [13]:
!pip install torchtext

In [0]:
import torchtext
from torchtext.vocab import Vectors
import torch
import numpy as np
import random

In [0]:
USE_CUDA = torch.cuda.is_available()

# random.seed(53113)
# np.random.seed(53113)
# torch.manual_seed(53113)
# if USE_CUDA:
#     torch.cuda.manual_seed(53113)

In [0]:
BATCH_SIZE = 32
EMBEDDING_SIZE = 650
MAX_VOCAB_SIZE = 50000

In [17]:
os.listdir('./')

['text8.zip', 'text8.dev.txt', 'text8.test.txt', 'text8.train.txt']

In [18]:
TEXT = torchtext.data.Field(lower=True)
train, val, test = torchtext.datasets.LanguageModelingDataset.splits(path=".", 
    train="text8.train.txt", validation="text8.dev.txt", test="text8.test.txt", text_field=TEXT)
TEXT.build_vocab(train, max_size=MAX_VOCAB_SIZE)
print("vocabulary size: {}".format(len(TEXT.vocab)))

VOCAB_SIZE = len(TEXT.vocab)
train_iter, val_iter, test_iter = torchtext.data.BPTTIterator.splits(
    (train, val, test), batch_size=BATCH_SIZE, device=-1, bptt_len=32, repeat=False, shuffle=True)

The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.
The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.
The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.


vocabulary size: 50002


In [0]:
import numpy as np

In [0]:
it = iter(train_iter)
batch = next(it)
# print(batch)
# batch.numpy()
print(' '.join([TEXT.vocab.itos[i] for i in batch.text[:,1].data]))

In [51]:
print(' '.join([TEXT.vocab.itos[i] for i in batch.text[:,1].data]))

combine in pairs and then group into trios of pairs which are the smallest visible units of matter this parallels with the structure of modern atomic theory in which pairs or triplets


In [59]:
TEXT.vocab.itos[batch.text[:,1][0]]

'combine'

In [57]:
for i in batch.text[:,1].data:
  print(i)

tensor(6271)
tensor(6)
tensor(3593)
tensor(4)
tensor(105)
tensor(191)
tensor(66)
tensor(29094)
tensor(3)
tensor(3593)
tensor(37)
tensor(27)
tensor(2)
tensor(4859)
tensor(2320)
tensor(1067)
tensor(3)
tensor(842)
tensor(34)
tensor(8292)
tensor(25)
tensor(2)
tensor(587)
tensor(3)
tensor(173)
tensor(2151)
tensor(214)
tensor(6)
tensor(37)
tensor(3593)
tensor(30)
tensor(25697)


In [0]:
import torch
import torch.nn as nn


class RNNModel(nn.Module):

    def __init__(self, rnn_type, ntoken, ninp, nhid, nlayers, dropout=0.5):
        super(RNNModel, self).__init__()
        self.drop = nn.Dropout(dropout)
        self.encoder = nn.Embedding(ntoken, ninp)
        if rnn_type in ['LSTM', 'GRU']:
            self.rnn = getattr(nn, rnn_type)(ninp, nhid, nlayers, dropout=dropout)
        else:
            try:
                nonlinearity = {'RNN_TANH': 'tanh', 'RNN_RELU': 'relu'}[rnn_type]
            except KeyError:
                raise ValueError( """An invalid option for `--model` was supplied,
                                 options are ['LSTM', 'GRU', 'RNN_TANH' or 'RNN_RELU']""")
            self.rnn = nn.RNN(ninp, nhid, nlayers, nonlinearity=nonlinearity, dropout=dropout)
        self.decoder = nn.Linear(nhid, ntoken)

        self.init_weights()

        self.rnn_type = rnn_type
        self.nhid = nhid
        self.nlayers = nlayers

    def init_weights(self):
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, input, hidden):
        emb = self.drop(self.encoder(input))
        output, hidden = self.rnn(emb, hidden)
        output = self.drop(output)
        decoded = self.decoder(output.view(output.size(0)*output.size(1), output.size(2)))
        return decoded.view(output.size(0), output.size(1), decoded.size(1)), hidden

    def init_hidden(self, bsz, requires_grad=True):
        weight = next(self.parameters())
        if self.rnn_type == 'LSTM':
            return (weight.new_zeros((self.nlayers, bsz, self.nhid), requires_grad=requires_grad),
                weight.new_zeros((self.nlayers, bsz, self.nhid), requires_grad=requires_grad))
        else:
            return weight.new_zeros((self.nlayers, bsz, self.nhid), requires_grad=requires_grad)

In [0]:
model = RNNModel("LSTM", VOCAB_SIZE, EMBEDDING_SIZE, EMBEDDING_SIZE, 2, dropout=0.5)
if USE_CUDA:
    model = model.cuda()

In [0]:
def evaluate(model, data):
  model.eval()
  total_loss = 0
  it = iter(data)
  total_count = 0
  with torch.no_grad():
    hidden = model.init_hidden(BATCH_SIZE,requires_grad=False)
    for i, batch in enumerate(it):
      data, target = batch.text, batch.target
      if USE_CUDA:
        data,target = data.cuda(), target.cuda()
      hidden = repackage_hidden(hidden)
      with torch.no_grad():
        output,hidden = model(data,hidden)
      loss = loss_fn(output.view(-1,VOCAB_SIZE),target.view(-1))
      total_count += np.multiply(*data.size())
      total_loss += loss.item()*np.multiply(*data.size())
  loss = total_loss / total_count
  model.train()
  return loss

In [0]:
# Remove this part
def repackage_hidden(h):
    """Wraps hidden states in new Tensors, to detach them from their history."""
    if isinstance(h, torch.Tensor):
        return h.detach()
    else:
        return tuple(repackage_hidden(v) for v in h)

In [0]:
loss_fn = nn.CrossEntropyLoss()
learning_rate = 0.001
optimizer = torch.optim.Adam(model.parameters(),lr = learning_rate)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer,0.5)

In [69]:
import copy
GRAD_CLIP = 1
NUM_EPOCH = 2

val_losses = []

for epoch in range(NUM_EPOCH):
  model.train()
  it = iter(train_iter)
  hidden = model.init_hidden(BATCH_SIZE)
  for i,batch in enumerate(it):
    data,target = batch.text, batch.target
    if USE_CUDA:
      data,target = data.cuda(),target.cuda()
    hidden = repackage_hidden(hidden)
    model.zero_grad()
    output,hidden = model(data,hidden)
    loss = loss_fn(output.view(-1,VOCAB_SIZE),target.view(-1))
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(),GRAD_CLIP)
    optimizer.step()
    if i % 1000 == 0:
      print('epoch',epoch,'iter',i,'loss',loss.item())
    if i % 10000 == 0:
      val_loss = evaluate(model,val_iter)

      if len(val_losses) == 0 or val_loss < min(val_losses):
        print('best model, val loss:',val_loss)
        torch.save(model.state_dict(),'lm-best.th')
      else:
        scheduler.step()
        optimizer = torch.optim.Adam(model.parameters(),lr=learning)
      val_losses.append(val_loss)

epoch 0 iter 0 loss 10.694487571716309
best model, val loss: 10.435706189865899
epoch 0 iter 1000 loss 6.393221378326416
epoch 0 iter 2000 loss 6.211528778076172
epoch 0 iter 3000 loss 6.217827796936035
epoch 0 iter 4000 loss 5.63140869140625
epoch 0 iter 5000 loss 5.8101301193237305
epoch 0 iter 6000 loss 5.867305278778076
epoch 0 iter 7000 loss 5.732617378234863
epoch 0 iter 8000 loss 5.874238014221191
epoch 0 iter 9000 loss 5.301486492156982
epoch 0 iter 10000 loss 5.932300090789795
best model, val loss: 5.258357186724829
epoch 0 iter 11000 loss 5.75202751159668
epoch 0 iter 12000 loss 5.48629903793335
epoch 0 iter 13000 loss 5.359623908996582
epoch 0 iter 14000 loss 5.5689873695373535
epoch 1 iter 0 loss 5.71110200881958
best model, val loss: 5.1006811082026635
epoch 1 iter 1000 loss 5.544424533843994
epoch 1 iter 2000 loss 5.3732218742370605
epoch 1 iter 3000 loss 5.607327938079834
epoch 1 iter 4000 loss 5.135415077209473
epoch 1 iter 5000 loss 5.469761371612549
epoch 1 iter 6000 